In [1]:
## Stardew Valley Optimization


# ~/Library/gurobi900
# python3 setup.py install 
import gurobipy as gp
from gurobipy import GRB


In [2]:
crops, cost, yeild_time, regrow, sell_price = gp.multidict({
    'blue_jaz': [30, 7, None, 50],
    'cauliflower': [80, 12, None, 175],
    'green_bean': [60, 10, 3, 40],
    'kale': [70, 6, None, 110],
    'parsnip': [20,4, None, 35],
    'potato': [50,6,None,80],
    'rhubarb': [100,13,None,220],
    'strawberry': [100,8,4,120],
    'tulip': [20,6,None,30],
    'unmilled_rice': [40,6,None,30]
})


In [3]:
cost['blue_jaz']

30

In [4]:
sell_price

{'blue_jaz': 50,
 'cauliflower': 175,
 'green_bean': 40,
 'kale': 110,
 'parsnip': 35,
 'potato': 80,
 'rhubarb': 220,
 'strawberry': 120,
 'tulip': 30,
 'unmilled_rice': 30}

In [5]:
def max_range_regrow(current_day, crop):
    if(current_day < yeild_time[crop]):
        return 0
    elif(current_day < yeild_time[crop] + regrow[crop]):
        return 1
    else:
        regrow_time = yeild_time[crop] + regrow[crop]
        count = 1
        while(current_day >= regrow_time):
            count +=1
            regrow_time += regrow[crop]
        return count

In [6]:
max_range_regrow(12,'green_bean')

1

In [17]:
# Create a new model
m = gp.Model("matrix1")
money = 2000
# Create variables
purchase =  []
planted = []
harvested = []
stored = []
money_in_days = []
amount_growing = []
days = 28
starting_money = 500
for i in range(days):
    money_in_days.append(m.addVar(name='money on ' + str(i)))
    purchase.append(m.addVars(crops,name='Purchased on ' + str(i)))
    planted.append(m.addVars(crops,name='planted on ' +str(i)))
    harvested.append(m.addVars(crops, name='harvested on ' +str(i)))
    stored.append(m.addVars(crops, name='stored on ' +str(i)))
    amount_growing.append(m.addVar(name='amount growing on ' +str(i)))

    
m.setObjective(money_in_days[days-1], GRB.MAXIMIZE)
    
for i in range(days):
    if(i==0):
        ## on day 0 i can spend up to starting_money
        m.addConstr(money_in_days[i] == starting_money - gp.quicksum(purchase[i][j]*cost[j] for j in crops))
#         m.addConstr(gp.quicksum(purchase[i][j] * cost[j] for j in crops) <= starting_money)
        ## on day 0 you can't harvest anything, what you store is what you buy and not plant
        ## what you plant is what you buy and not store
        for j in crops:
            m.addConstr(stored[i][j] == purchase[i][j] - planted[i][j])
            m.addConstr(planted[i][j] == purchase[i][j] - stored[i][j])
            m.addConstr(harvested[i][j] == 0)
    else:
        m.addConstr(amount_growing[i] == gp.quicksum(gp.quicksum(planted[k][j] - harvested[k][j] for j in crops) 
                                                     for k in range(i)))
        m.addConstr(2*amount_growing[i] <= 270)
        m.addConstr(money_in_days[i] == money_in_days[i-1] - gp.quicksum(purchase[i][j]*cost[j] for j in crops)
                   + gp.quicksum(harvested[i][j] * sell_price[j] for j in crops))
        m.addConstr(gp.quicksum(purchase[i][j] for j in crops) <= money_in_days[i] + 
                    gp.quicksum(harvested[i][j] for j in crops))
        for j in crops:
            m.addConstr(stored[i][j] == stored[i-1][j] + purchase[i][j] - planted[i][j])
            m.addConstr(planted[i][j] <= stored[i-1][j] + purchase[i][j])
            if(yeild_time[j] > i):
                m.addConstr(harvested[i][j] == 0)
            else:
                if(regrow[j] is not None):
                    m.addConstr(harvested[i][j] == gp.quicksum(planted[i-yeild_time[j]-k*regrow[j]][j] 
                                                               for k in range(max_range_regrow(i,j))))
                else:
                    m.addConstr(harvested[i][j] == (planted[i-yeild_time[j]][j]))

m.optimize()
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 507 rows, 630 columns and 4008 nonzeros
Model fingerprint: 0xc70f33c4
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+02]
Presolve removed 220 rows and 217 columns
Presolve time: 0.03s
Presolved: 287 rows, 413 columns, 2108 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0831319e+05   3.939561e+03   0.000000e+00      0s
     166    2.6795389e+03   0.000000e+00   0.000000e+00      0s

Solved in 166 iterations and 0.05 seconds
Optimal objective  2.679538907e+03
money on 0 0
Purchased on 0[blue_jaz] 0
Purchased on 0[cauliflower] 0
Purchased on 0[green_bean] 0
Purchased on 0[kale] 0
Purchased on 0[parsnip] 25
Purchased on 0[potato] 0
Purchased on 0[rhubarb] 0
Purchased on 0[strawberry] 0
Purchased on 0[tulip] 0
Purchased on 0[unmilled_rice] 0
planted on 0[blue_jaz] 0
p